In [1]:
#!pip install leafmap

In [2]:
#!pip install pmdarima

In [42]:
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px     
import numpy as np
import requests
import kaggle
import geocoder
import folium
import leafmap.foliumap as leafmap
from geopy.geocoders import Nominatim


In [43]:
df = pd.read_csv("shootingdefinitief.csv")

In [44]:
df = df.drop(["Unnamed: 0"], axis=1)

In [45]:
df['year'] = pd.DatetimeIndex(df['date']).year

In [46]:
df.drop(["latitude_city", "longitude_city",
             "latitude_state", "longitude_state",
             "latitude_state_1", "longitude_state_1"],
             axis = 1, inplace=True)

# Data cleaning

In [47]:
df

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,arms_category,year,lon state,lat state
0,3,Tim Elliot,2015-01-02,shot,gun,53.0,M,Asian,Shelton,Washington,True,attack,Not fleeing,False,Guns,2015,-77.036543,38.89504
1,4,Lewis Lee Lembke,2015-01-02,shot,gun,47.0,M,White,Aloha,Oregon,False,attack,Not fleeing,False,Guns,2015,-120.737257,43.97928
2,5,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.0,M,Hispanic,Wichita,Kansas,False,other,Not fleeing,False,Unarmed,2015,-98.582187,38.27312
3,8,Matthew Hoffman,2015-01-04,shot,toy weapon,32.0,M,White,San Francisco,California,True,attack,Not fleeing,False,Other unusual objects,2015,-118.755997,36.70146
4,9,Michael Rodriguez,2015-01-04,shot,nail gun,39.0,M,Hispanic,Evans,Colorado,False,attack,Not fleeing,False,Piercing objects,2015,-105.607716,38.72518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4890,5916,Rayshard Brooks,2020-06-12,shot,Taser,27.0,M,Black,Atlanta,Georgia,False,attack,Foot,True,Electrical devices,2020,-83.113737,32.32938
4891,5925,Caine Van Pelt,2020-06-12,shot,gun,23.0,M,Black,Crown Point,Indiana,False,attack,Car,False,Guns,2020,-86.174693,40.32701
4892,5918,Hannah Fizer,2020-06-13,shot,unarmed,25.0,F,White,Sedalia,Missouri,False,other,Not fleeing,False,Unarmed,2020,-92.561787,38.76048
4893,5921,William Slyter,2020-06-13,shot,gun,22.0,M,White,Kansas City,Missouri,False,other,Other,False,Guns,2020,-92.561787,38.76048


# Doden per state

In [48]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'Washington',
        'DE': 'Delaware',
        'FL' : 'Florida',
        'GA' : 'Georgia',
        'HI' : 'Hawaii',
        'ID' : 'Idaho',
        'IL' : 'Illinois',
        'IN' : 'Indiana',
        'IA' : 'Iowa',
        'KS' : 'Kansas',
        'KY' : 'Kentucky',
        'LA' : 'Louisiana',
        'ME' : 'Maine',
        'MD' : 'Maryland',
        'MA' : 'Massachusetss',
        'MI' : 'Michigan',
        'MN' : 'Minnesota',
        'MS' : 'Mississippi',
        'MO' : 'Missouri',
        'MT' : 'Montana',
        'NE' : 'Nebraska',
        'NV' : 'Nevada',
        'NH' : 'New Hampshire',
        'NJ' : 'New Jersey',
        'NM' : 'New Mexico',
        'NY' : 'New York',
        'NC' : 'North Carolina',
        'ND' : 'North Dakota',
        'OH' : 'Ohio',
        'OK' : 'Oklahoma',
        'OR' : 'Oregon',
        'PA' : 'Pennsylvania',
        'RI' : 'Rhode Island',
        'SC' : 'South Carolina',
        'SD' : 'South Dakota',
        'TN' : 'Tennessee',
        'TX' : 'Texas',
        'UT' : 'Utah',
        'VT' : 'Vermont',
        'VA' : 'Virginia',
        'WA' : 'Washington',
        'WV' : 'West Virginia',
        'WI' : 'Wisconsin',
        'WY' : 'Wyoming'}

In [49]:
df["state"] = df["state"].replace(states)

# Achterhalen lon lat (state)

In [50]:
df.head()

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,arms_category,year,lon state,lat state
0,3,Tim Elliot,2015-01-02,shot,gun,53.0,M,Asian,Shelton,Washington,True,attack,Not fleeing,False,Guns,2015,-77.036543,38.89504
1,4,Lewis Lee Lembke,2015-01-02,shot,gun,47.0,M,White,Aloha,Oregon,False,attack,Not fleeing,False,Guns,2015,-120.737257,43.97928
2,5,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.0,M,Hispanic,Wichita,Kansas,False,other,Not fleeing,False,Unarmed,2015,-98.582187,38.27312
3,8,Matthew Hoffman,2015-01-04,shot,toy weapon,32.0,M,White,San Francisco,California,True,attack,Not fleeing,False,Other unusual objects,2015,-118.755997,36.70146
4,9,Michael Rodriguez,2015-01-04,shot,nail gun,39.0,M,Hispanic,Evans,Colorado,False,attack,Not fleeing,False,Piercing objects,2015,-105.607716,38.72518


In [51]:
#with requests.Session() as session:
#    for state in df["state"]:
#        coo = geocoder.osm(state, session=session)
#        if coo:
#            df.loc[df["state"] == state, "lon state"] = round(coo.osm["x"], 7)
#            df.loc[df["state"] == state, "lat state"] = round(coo.osm["y"], 5)

In [52]:
#df.to_csv("shootingdefinitief.csv")

In [53]:
mapdata = pd.DataFrame(df.groupby(["state", "lat state", "lon state"])["manner_of_death"].count())
mapdata = mapdata.reset_index()
mapdata

,state,lat state,lon state,manner_of_death
0,Alabama,33.25888,-86.829534,95
1,Alaska,64.44596,-149.680909,36
2,Arizona,34.39534,-111.763275,222
3,Arkansas,35.20489,-92.447911,73
4,California,36.70146,-118.755997,701
5,Colorado,38.72518,-105.607716,168
6,Connecticut,41.65002,-72.734216,20
7,Delaware,38.69205,-75.401331,10
8,Florida,27.75677,-81.463983,324
9,Georgia,32.32938,-83.113737,161


# Achterhalen lon lat (city) 

In [54]:
city = pd.read_csv("uscities.csv")
city.drop(["id", "zips",
             "ranking", "timezone",
             "incorporated", "military",
          "source", "density", "population", "county_name","county_fips"],
             axis = 1, inplace=True)
city = city.rename(columns={"lat":"lat city", "lng":"lon city", "state_name":"state"})
city

,city,city_ascii,state_id,state,lat city,lon city
0,New York,New York,NY,New York,40.6943,-73.9249
1,Los Angeles,Los Angeles,CA,California,34.1141,-118.4068
2,Chicago,Chicago,IL,Illinois,41.8375,-87.6866
3,Miami,Miami,FL,Florida,25.7840,-80.2101
4,Dallas,Dallas,TX,Texas,32.7935,-96.7667
...,...,...,...,...,...,...
30404,Drummond,Drummond,ID,Idaho,43.9996,-111.3433
30405,Lost Springs,Lost Springs,WY,Wyoming,42.7652,-104.9255
30406,Provo,Provo,SD,South Dakota,43.1937,-103.8329
30407,Goldcreek,Goldcreek,MT,Montana,46.5838,-112.9284


In [55]:
city.columns

Index(['city', 'city_ascii', 'state_id', 'state', 'lat city', 'lon city'], dtype='object')

In [56]:
df = pd.merge(df, city, how="left", left_on=["city", "state"], right_on=["city", "state"])

In [57]:
df.drop(["state_id", "city_ascii"], axis=1, inplace=True)

In [58]:
df.dropna()

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,arms_category,year,lon state,lat state,lat city,lon city
0,3,Tim Elliot,2015-01-02,shot,gun,53.0,M,Asian,Shelton,Washington,True,attack,Not fleeing,False,Guns,2015,-77.036543,38.89504,47.2186,-123.1121
1,4,Lewis Lee Lembke,2015-01-02,shot,gun,47.0,M,White,Aloha,Oregon,False,attack,Not fleeing,False,Guns,2015,-120.737257,43.97928,45.4920,-122.8725
2,5,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.0,M,Hispanic,Wichita,Kansas,False,other,Not fleeing,False,Unarmed,2015,-98.582187,38.27312,37.6895,-97.3443
3,8,Matthew Hoffman,2015-01-04,shot,toy weapon,32.0,M,White,San Francisco,California,True,attack,Not fleeing,False,Other unusual objects,2015,-118.755997,36.70146,37.7558,-122.4449
4,9,Michael Rodriguez,2015-01-04,shot,nail gun,39.0,M,Hispanic,Evans,Colorado,False,attack,Not fleeing,False,Piercing objects,2015,-105.607716,38.72518,40.3660,-104.7390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4893,5916,Rayshard Brooks,2020-06-12,shot,Taser,27.0,M,Black,Atlanta,Georgia,False,attack,Foot,True,Electrical devices,2020,-83.113737,32.32938,33.7628,-84.4220
4894,5925,Caine Van Pelt,2020-06-12,shot,gun,23.0,M,Black,Crown Point,Indiana,False,attack,Car,False,Guns,2020,-86.174693,40.32701,41.4143,-87.3457
4895,5918,Hannah Fizer,2020-06-13,shot,unarmed,25.0,F,White,Sedalia,Missouri,False,other,Not fleeing,False,Unarmed,2020,-92.561787,38.76048,38.7059,-93.2340
4896,5921,William Slyter,2020-06-13,shot,gun,22.0,M,White,Kansas City,Missouri,False,other,Other,False,Guns,2020,-92.561787,38.76048,39.1238,-94.5541


In [28]:
#df.to_csv("shootings_data.csv")